### Peewee ORM library

#### This tutorial is based on http://docs.peewee-orm.com/en/latest/peewee/quickstart.html
---

Model classes, fields and model instances all map to database concepts:

| Object         | Corresponds to… |
| :------------: | :-------------: |
| Model class    | Database table  |
| Field instance | Column on a table |
| Model instance | Row in a database table |

 See example:

```
class Person(Model):
    name = CharField()
    birthday = DateField()

    class Meta:
        database = db # This model uses the "people.db" database.
```

In [1]:
# uncomment this library for installing the library
#!pip install peewee

In [2]:
# import library
from peewee import *

In [3]:
# let's begin by creating a database
db = SqliteDatabase('people.db')

In [4]:
# Peewee will automatically infer the database table name from the name of the class. 
# You can override the default name by specifying a `table_name` attribute in the inner `“Meta” class` 
# (alongside the `database` attribute.
class Person(Model):
    name = CharField()
    birthday = DateField()

    class Meta:
        database = db # This model uses the "people.db" database.

There are lots of field types suitable for storing various types of data. Peewee handles converting between pythonic values and those used by the database, so you can use Python types in your code without having to worry.

Things get interesting when we set up **relationships between models** using *foreign key* relationships. 

In [5]:
# create a second model and expressing a foreign key relationship
# have a look at the [documentation](http://docs.peewee-orm.com/en/latest/peewee/models.html#foreignkeyfield-back-references)
class Pet(Model):
    owner = ForeignKeyField(Person, backref='pets')
    name = CharField()
    animal_type = CharField()

    class Meta:
        database = db # this model uses the "people.db" database

In [6]:
# explicitly connect to the database
# not necessary, but recommended for checking connection errors prior to executing any queries
db.connect()

In [7]:
# # create the tables with the appropriate columns, indexes, sequences, and foreign key constraints
db.create_tables([Person, Pet])

#### Storing data

We can use two methods to insert data into a table (model):
- `save()`: it can be used to insert a new row or updating an existing row
- `create()`: it can be used to insert a new row and return an instance to that row

In [8]:
from datetime import date

# inserting data using save()
uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15))
uncle_bob.save() # Bob is now stored in the database

In [9]:
# inserting data using create()
grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1))
herb = Person.create(name='Herb', birthday=date(1950, 5, 5))

#### Updating data

In [10]:
# to update a row, modify the model instance and call save() to persist the changes. 
# Here we will change Grandma’s name and then save the changes in the database:
grandma.name = 'Grandma L.'
grandma.save()  

1

In [11]:
# now we have stored 3 people in the database. Let’s give them some pets. 
bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')

In [12]:
# if you want to remove some instance
herb_mittens.delete_instance() # he had a great life

1

In [13]:
# if you want to reassing some instance (i.e. modify some content)
herb_fido.owner = uncle_bob
herb_fido.save()

1

### Retrieving/Querying data

Let’s retrieve Grandma’s record from the database. To get a single record from the database, use `Select.get()`:

In [14]:
# retrieving one single instance
grandma = Person.select().where(Person.name == 'Grandma L.').get()
print("Name:", grandma.name, "Birthday:", grandma.birthday)

Name: Grandma L. Birthday: 1935-03-01


In [15]:
# alternatively, we can use the equivalent shorthand Model.get():
grandma = Person.get(Person.name == 'Grandma L.')
print("Name:", grandma.name, "Birthday:", grandma.birthday)

Name: Grandma L. Birthday: 1935-03-01


In [16]:
# we can iterate over all instances of a given model
for person in Person.select():
    print("Person name:", person.name)

Person name: Bob
Person name: Grandma L.
Person name: Herb


In [17]:
# let’s list all the cats and their owner’s name:
query = Pet.select().where(Pet.animal_type == 'cat')
for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


According to Peewee documentation, the previous query suffers from the [**N+1 problem**](http://docs.peewee-orm.com/en/latest/peewee/relationships.html#nplusone), which represents an extra table (model) that needs to be queried for retrieving the `pet.owner.name` information. This can be avoided by selecting both Pet and Person, and adding a join.

In [18]:
query = (Pet
         .select(Pet, Person)
         .join(Person)
         .where(Pet.animal_type == 'cat'))

for pet in query:
    print("Pet name:", pet.name, "Owner:", pet.owner.name)

Pet name: Kitty Owner: Bob
Pet name: Mittens Jr Owner: Herb


In [19]:
# let's retrieve all pets owned by Bob
for pet in Pet.select().join(Person).where(Person.name == 'Bob'):
    print(pet.name)

Kitty
Fido


In [20]:
# alternatively, as we already have an object to represent Bob, we can do this instead:
for pet in Pet.select().where(Pet.owner == uncle_bob):
    print(pet.name)

Kitty
Fido


### Sorting

Let’s make sure these are sorted alphabetically by adding an `order_by()` clause:

In [21]:
for pet in Pet.select().where(Pet.owner == uncle_bob).order_by(Pet.name):
    print(pet.name)

Fido
Kitty


In [22]:
# let’s list all the people now, youngest to oldest:
for person in Person.select().order_by(Person.birthday.desc()):
    print(person.name, person.birthday)

Bob 1960-01-15
Herb 1950-05-05
Grandma L. 1935-03-01


### Combining filter extensions

Peewee supports arbitrarily-nested expressions. Let’s get all the people whose birthday was either:

- before 1940 (`grandma`)
- after 1959 (`bob`)

In [23]:
d1940 = date(1940, 1, 1)
d1960 = date(1960, 1, 1)
query = (Person
         .select()
         .where((Person.birthday < d1940) | (Person.birthday > d1960)))

for person in query:
    print("Name:", person.name, "Birthday:", person.birthday)

Name: Bob Birthday: 1960-01-15
Name: Grandma L. Birthday: 1935-03-01


Now let’s do the opposite. People whose birthday is between 1940 and 1960:

In [24]:
query = (Person
         .select()
         .where(Person.birthday.between(d1940, d1960)))

for person in query:
    print(person.name, person.birthday)

Herb 1950-05-05


### Aggregates and prefetch

In [25]:
# now let’s list all the people and how many pets they have:
for person in Person.select():
    print("Name:", person.name, "Number of pets:", person.pets.count(), 'pets')

Name: Bob Number of pets: 2 pets
Name: Grandma L. Number of pets: 0 pets
Name: Herb Number of pets: 1 pets


Once again we’ve run into a classic example of **N+1 query behavior**. In this case, we’re executing an additional query for every `Person` returned by the original SELECT! We can avoid this by performing a JOIN and using a SQL function to aggregate the results.

Peewee provides a magical helper `fn()`, which can be used to call any SQL function. In the example below, `fn.COUNT(Pet.id).alias('pet_count')` would be translated into `COUNT(pet.id) AS pet_count`.

In [26]:
query = (Person
         .select(Person, fn.COUNT(Pet.id).alias('pet_count'))
         .join(Pet, JOIN.LEFT_OUTER)  # include people without pets.
         .group_by(Person)
         .order_by(Person.name))

for person in query:
    # "pet_count" becomes an attribute on the returned model instances.
    print("Name:", person.name, "Number of pets:", person.pet_count, 'pets')

Name: Bob Number of pets: 2 pets
Name: Grandma L. Number of pets: 0 pets
Name: Herb Number of pets: 1 pets


`[from the Peewee documentation]`

Now let’s list all the people and the names of all their pets. As you may have guessed, this could easily turn into another **N+1** situation if we’re not careful.

Before diving into the code, consider how this example is different from the earlier example where we listed all the pets and their owner’s name. A pet can only have one owner, so when we performed the join from `Pet` to `Person`, there was always going to be a single match. The situation is different when we are joining from `Person` to `Pet` because a person may have zero pets or they may have several pets. Because we’re using a relational databases, if we were to do a join from `Person` to `Pet` then every person with multiple pets would be repeated, once for each pet.

In [27]:
query = (Person
         .select(Person, Pet)
         .join(Pet, JOIN.LEFT_OUTER)
         .order_by(Person.name, Pet.name))

for person in query:
    # We need to check if they have a pet instance attached, since not all people have pets.
    if hasattr(person, 'pet'):
        print(person.name, person.pet.name)
    else:
        print(person.name, 'no pets')

Bob Fido
Bob Kitty
Grandma L. no pets
Herb Mittens Jr


The following example uses a special method called `prefetch()` to accommodate the more common (and intuitive) workflow of listing a person and attaching a list of that person’s pets (and thus avoiding duplicates).

In [28]:
query = Person.select().order_by(Person.name).prefetch(Pet)

for person in query:
    print(person.name)
    for pet in person.pets:
        print('  *', pet.name)

Bob
  * Kitty
  * Fido
Grandma L.
Herb
  * Mittens Jr


### SQL functions

Just an example of how SQL queries can be used. Have a look at the [documentation](http://docs.peewee-orm.com/en/latest/peewee/querying.html#querying) for more complex queries.

This will find all people whose names start with either an upper or lower-case G:

In [29]:
expression = fn.Lower(fn.Substr(Person.name, 1, 1)) == 'g'
for person in Person.select().where(expression):
    print(person.name)

Grandma L.


In [30]:
# closing the connection
db.close()

True